In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle("data/urfunny.pkl")
df

,sentence,features,intervals,timings,split,file_id,root,label,context,timings2,proc_intervals_4,audio_path,video_path,test,video_timings,audio_timings,speaker
0,[finally in desperation they went and they con...,"[[finally, in, desperation, they, went, and, t...","[[[0.0, 0.339], [0.339, 0.459], [0.459, 1.157]...","(24.933, 28.206)",test,11318,1017,CONTEXT,True,"[(0.0, 4.37), (5.019, 12.252), (13.519, 14.717...","(21.022, 28.206)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(24.933, 28.206)","(0.0, 28.206)",None
1,they gave it back to the wise old woman,"[they, gave, it, back, to, the, wise, old, woman]","[[28.415, 28.475], [28.475, 28.565], [28.565, ...","(28.415, 29.273)",test,11318,1017,1,False,"(28.415, 29.273)","(28.415, 29.273)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(28.415, 29.273)","(28.415, 29.273)",None
2,[the most fundamental way in which the third s...,"[[the, most, fundamental, way, in, which, the,...","[[[0.0, 0.06], [0.06, 0.299], [0.319, 1.427], ...","(21.92, 25.841)",test,11319,1017,CONTEXT,True,"[(0.0, 7.273), (8.022, 15.475), (16.233, 21.57...","(21.92, 25.841)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(21.92, 25.841)","(0.0, 25.841)",None
3,and as the saying goes when angry you will mak...,"[and, as, the, saying, goes, when, angry, you,...","[[26.889, 27.039], [27.039, 27.188], [27.188, ...","(26.889, 29.772)",test,11319,1017,1,False,"(26.889, 29.772)","(26.889, 29.772)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(26.889, 29.772)","(26.889, 29.772)",None
4,[i tried to remember to go to the balcony],"[[i, tried, to, remember, to, go, to, the, bal...","[[[0.0, 0.18], [0.18, 0.499], [0.499, 0.559], ...","(0.0, 2.365)",test,11321,1017,CONTEXT,True,"[(0.0, 2.365)]","(0.0, 2.365)",../../data/urfunny2_audio/11321.wav,../../data/urfunny2_videos/11321.mp4,lol,"(0.0, 2.365)","(0.0, 2.365)",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20327,so i found that they had a friend a local acco...,"[so, i, found, that, they, had, a, friend, a, ...","[[56.142, 56.482], [56.482, 56.781], [56.881, ...","(56.142, 61.69)",val,10945,949,0,False,"(56.142, 61.69)","(56.142, 61.69)",../../data/urfunny2_audio/10945.wav,../../data/urfunny2_videos/10945.mp4,lol,"(56.142, 61.69)","(56.142, 61.69)",None
20328,[so their scores had gone up from zero to perc...,"[[so, their, scores, had, gone, up, from, zero...","[[[0.0, 0.359], [0.359, 0.489], [0.489, 0.878]...","(24.644, 30.062)",val,10946,949,CONTEXT,True,"[(0.0, 5.777), (6.146, 6.824), (8.381, 13.928)...","(24.644, 30.062)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(24.644, 30.062)","(0.0, 30.062)",None
20329,just say to them cool,"[just, say, to, them, cool]","[[31.389, 31.558], [31.558, 31.708], [31.708, ...","(31.389, 32.177)",val,10946,949,0,False,"(31.389, 32.177)","(31.389, 32.177)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(31.389, 32.177)","(31.389, 32.177)",None
20330,"[the worst in, they used everything that they ...","[[the, worst, in], [they, used, everything, th...","[[[0.0, 0.06], [0.06, 0.2], [0.409, 0.559]], [...","(33.354, 37.245)",val,10948,949,CONTEXT,True,"[(0.0, 0.559), (1.896, 5.927), (7.982, 11.963)...","(11.903, 37.245)",../../data/urfunny2_audio/10948.wav,../../data/urfunny2_videos/10948.mp4,lol,"(33.354, 37.245)","(4.929, 37.245)",None


In [41]:
def process_intervals(intervals, limit = 4):
    # Initialize the running total and the limit
    # print(intervals , flush = True)
    if  len(intervals)  == 0:
        return (0 , 500)    
    if isinstance(intervals[0][0] , list) or isinstance(intervals[0][0] , np.ndarray):
        
        running_total = 0

        # Initialize the list to store the intervals that fit within the limit
        limited_intervals = []

        # Go through the intervals in reverse order
        for idx_i , i in enumerate(reversed(range(len(intervals)))):
            sublist = intervals[i]
            for j , interval in enumerate(reversed(sublist)):
                # Calculate the absolute difference between end and start
                time_diff = abs(interval[1] - interval[0])
                # If adding this time difference to the running total would make it greater than the limit
                if running_total + time_diff > limit and idx_i == 0 and j == 0:
                    # if on the first iteration the first interval is already greater than the limit
                    limited_intervals.append(interval)
                    break
                elif running_total + time_diff > limit:
                    # Stop the loop
                    break
                
                # Add the time difference to the running total
                running_total += time_diff
                
                # Add the interval to the list
                limited_intervals.append(interval)
            
            # If we haven't reached the limit yet, add the gap to the next sublist (if there is one)
            if i > 0 and running_total < limit:
                gap = intervals[i][0][0] - intervals[i-1][-1][1]
                running_total += gap

        # Reverse the list of intervals to get them in the original order
        limited_intervals = list(reversed(limited_intervals))

        return (limited_intervals[0][0], limited_intervals[-1][1])
    else:
        return (intervals[0][0], intervals[-1][1])

In [42]:
from tqdm import tqdm
tqdm.pandas()

df['timings']  = df['intervals'].progress_apply( lambda x: process_intervals(x , limit = 4))
df['video_timings']  = df['intervals'].progress_apply( lambda x: process_intervals(x , limit = 4))
df['audio_timings']  = df['intervals'].progress_apply( lambda x: process_intervals(x , limit = 29))
df

  0%|          | 0/20332 [00:00<?, ?it/s]

100%|██████████| 20332/20332 [00:00<00:00, 102706.80it/s]


,sentence,features,intervals,timings,split,file_id,root,label,context,timings2,proc_intervals_4,audio_path,video_path,test,video_timings,audio_timings,speaker,absolute
0,[finally in desperation they went and they con...,"[[finally, in, desperation, they, went, and, t...","[[[0.0, 0.339], [0.339, 0.459], [0.459, 1.157]...","(24.933, 28.206)",test,11318,1017,CONTEXT,True,"[(0.0, 4.37), (5.019, 12.252), (13.519, 14.717...","(21.022, 28.206)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(24.933, 28.206)","(0.0, 28.206)",None,28.206
1,they gave it back to the wise old woman,"[they, gave, it, back, to, the, wise, old, woman]","[[28.415, 28.475], [28.475, 28.565], [28.565, ...","(28.415, 29.273)",test,11318,1017,1,False,"(28.415, 29.273)","(28.415, 29.273)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(28.415, 29.273)","(28.415, 29.273)",None,0.858
2,[the most fundamental way in which the third s...,"[[the, most, fundamental, way, in, which, the,...","[[[0.0, 0.06], [0.06, 0.299], [0.319, 1.427], ...","(21.92, 25.841)",test,11319,1017,CONTEXT,True,"[(0.0, 7.273), (8.022, 15.475), (16.233, 21.57...","(21.92, 25.841)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(21.92, 25.841)","(0.0, 25.841)",None,25.841
3,and as the saying goes when angry you will mak...,"[and, as, the, saying, goes, when, angry, you,...","[[26.889, 27.039], [27.039, 27.188], [27.188, ...","(26.889, 29.772)",test,11319,1017,1,False,"(26.889, 29.772)","(26.889, 29.772)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(26.889, 29.772)","(26.889, 29.772)",None,2.883
4,[i tried to remember to go to the balcony],"[[i, tried, to, remember, to, go, to, the, bal...","[[[0.0, 0.18], [0.18, 0.499], [0.499, 0.559], ...","(0.0, 2.365)",test,11321,1017,CONTEXT,True,"[(0.0, 2.365)]","(0.0, 2.365)",../../data/urfunny2_audio/11321.wav,../../data/urfunny2_videos/11321.mp4,lol,"(0.0, 2.365)","(0.0, 2.365)",None,2.365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20327,so i found that they had a friend a local acco...,"[so, i, found, that, they, had, a, friend, a, ...","[[56.142, 56.482], [56.482, 56.781], [56.881, ...","(56.142, 61.69)",val,10945,949,0,False,"(56.142, 61.69)","(56.142, 61.69)",../../data/urfunny2_audio/10945.wav,../../data/urfunny2_videos/10945.mp4,lol,"(56.142, 61.69)","(56.142, 61.69)",None,5.548
20328,[so their scores had gone up from zero to perc...,"[[so, their, scores, had, gone, up, from, zero...","[[[0.0, 0.359], [0.359, 0.489], [0.489, 0.878]...","(24.644, 30.062)",val,10946,949,CONTEXT,True,"[(0.0, 5.777), (6.146, 6.824), (8.381, 13.928)...","(24.644, 30.062)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(24.644, 30.062)","(0.0, 30.062)",None,30.062
20329,just say to them cool,"[just, say, to, them, cool]","[[31.389, 31.558], [31.558, 31.708], [31.708, ...","(31.389, 32.177)",val,10946,949,0,False,"(31.389, 32.177)","(31.389, 32.177)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(31.389, 32.177)","(31.389, 32.177)",None,0.788
20330,"[the worst in, they used everything that they ...","[[the, worst, in], [they, used, everything, th...","[[[0.0, 0.06], [0.06, 0.2], [0.409, 0.559]], [...","(33.354, 37.245)",val,10948,949,CONTEXT,True,"[(0.0, 0.559), (1.896, 5.927), (7.982, 11.963)...","(11.903, 37.245)",../../data/urfunny2_audio/10948.wav,../../data/urfunny2_videos/10948.mp4,lol,"(33.354, 37.245)","(4.929, 37.245)",None,32.316


In [43]:
df.to_pickle("data/urfunny.pkl")

True

In [44]:
df['absolute'] = df['audio_timings'].apply(lambda x: x[1] - x[0])


In [5]:
jdx = 6
df['intervals'].iloc[jdx] , df['audio_timings'].iloc[jdx] , df['video_timings'].iloc[jdx] , df['absolute'] .iloc[jdx]

(array([[[0.   , 0.11 ],
         [0.11 , 0.399],
         [0.399, 0.729],
         [0.729, 0.968],
         [1.068, 1.417]],
 
        [[1.627, 1.686],
         [1.686, 1.876],
         [1.876, 2.165],
         [2.165, 2.345],
         [2.345, 3.003]]]),
 (array([0.  , 0.11]), array([1.686, 1.876])),
 (array([0.  , 0.11]), array([1.686, 1.876])),
 array([1.686, 1.766]))

In [9]:
df[ len(df['absolute'])  == 1 ]

KeyError: False

In [24]:
df[df['absolute'].apply(lambda x: not isinstance(x, int))]

,sentence,features,intervals,timings,split,file_id,root,label,context,timings2,proc_intervals_4,audio_path,video_path,test,video_timings,audio_timings,speaker,absolute
0,[finally in desperation they went and they con...,"[[finally, in, desperation, they, went, and, t...","[[[0.0, 0.339], [0.339, 0.459], [0.459, 1.157]...","(24.933, 28.206)",test,11318,1017,CONTEXT,True,"[(0.0, 4.37), (5.019, 12.252), (13.519, 14.717...","(21.022, 28.206)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(24.933, 28.206)","(0.0, 28.206)",None,28.206
1,they gave it back to the wise old woman,"[they, gave, it, back, to, the, wise, old, woman]","[[28.415, 28.475], [28.475, 28.565], [28.565, ...","(28.415, 29.273)",test,11318,1017,1,False,"(28.415, 29.273)","(28.415, 29.273)",../../data/urfunny2_audio/11318.wav,../../data/urfunny2_videos/11318.mp4,lol,"(28.415, 29.273)","(28.415, 29.273)",None,0.858
2,[the most fundamental way in which the third s...,"[[the, most, fundamental, way, in, which, the,...","[[[0.0, 0.06], [0.06, 0.299], [0.319, 1.427], ...","(21.92, 25.841)",test,11319,1017,CONTEXT,True,"[(0.0, 7.273), (8.022, 15.475), (16.233, 21.57...","(21.92, 25.841)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(21.92, 25.841)","(0.0, 25.841)",None,25.841
3,and as the saying goes when angry you will mak...,"[and, as, the, saying, goes, when, angry, you,...","[[26.889, 27.039], [27.039, 27.188], [27.188, ...","(26.889, 29.772)",test,11319,1017,1,False,"(26.889, 29.772)","(26.889, 29.772)",../../data/urfunny2_audio/11319.wav,../../data/urfunny2_videos/11319.mp4,lol,"(26.889, 29.772)","(26.889, 29.772)",None,2.883
4,[i tried to remember to go to the balcony],"[[i, tried, to, remember, to, go, to, the, bal...","[[[0.0, 0.18], [0.18, 0.499], [0.499, 0.559], ...","(0.0, 2.365)",test,11321,1017,CONTEXT,True,"[(0.0, 2.365)]","(0.0, 2.365)",../../data/urfunny2_audio/11321.wav,../../data/urfunny2_videos/11321.mp4,lol,"(0.0, 2.365)","(0.0, 2.365)",None,2.365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20327,so i found that they had a friend a local acco...,"[so, i, found, that, they, had, a, friend, a, ...","[[56.142, 56.482], [56.482, 56.781], [56.881, ...","(56.142, 61.69)",val,10945,949,0,False,"(56.142, 61.69)","(56.142, 61.69)",../../data/urfunny2_audio/10945.wav,../../data/urfunny2_videos/10945.mp4,lol,"(56.142, 61.69)","(56.142, 61.69)",None,5.548
20328,[so their scores had gone up from zero to perc...,"[[so, their, scores, had, gone, up, from, zero...","[[[0.0, 0.359], [0.359, 0.489], [0.489, 0.878]...","(24.644, 30.062)",val,10946,949,CONTEXT,True,"[(0.0, 5.777), (6.146, 6.824), (8.381, 13.928)...","(24.644, 30.062)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(24.644, 30.062)","(0.0, 30.062)",None,30.062
20329,just say to them cool,"[just, say, to, them, cool]","[[31.389, 31.558], [31.558, 31.708], [31.708, ...","(31.389, 32.177)",val,10946,949,0,False,"(31.389, 32.177)","(31.389, 32.177)",../../data/urfunny2_audio/10946.wav,../../data/urfunny2_videos/10946.mp4,lol,"(31.389, 32.177)","(31.389, 32.177)",None,0.788
20330,"[the worst in, they used everything that they ...","[[the, worst, in], [they, used, everything, th...","[[[0.0, 0.06], [0.06, 0.2], [0.409, 0.559]], [...","(33.354, 37.245)",val,10948,949,CONTEXT,True,"[(0.0, 0.559), (1.896, 5.927), (7.982, 11.963)...","(11.903, 37.245)",../../data/urfunny2_audio/10948.wav,../../data/urfunny2_videos/10948.mp4,lol,"(33.354, 37.245)","(4.929, 37.245)",None,32.316


(53.548, 56.821)

In [ ]:
isinstance(a[0][0] , list)

In [ ]:
process_intervals(df['intervals'].iloc[1])